<a href="https://colab.research.google.com/github/DheerajAluru/Machine_Learning_Course/blob/main/Transfer_learning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Loading necessary packages

In [1]:

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input,Conv2D,MaxPool2D,Activation,Dropout,Flatten
from tensorflow.keras.models import Model,Sequential
import random as rn
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from glob import glob
import matplotlib.pyplot as plt

tf.__version__

'2.4.1'

In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

##### Downloading data

In [ ]:
!wget --header="Host: doc-0c-80-docs.googleusercontent.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://drive.google.com/drive/u/1/folders/1eY4pTqtGtXSAYVzaHwWAK2Cq4bHb1Zkq" --header="Cookie: AUTH_g91r08ja0v2lklj0jvdr2avi84sjt90s_nonce=nacpsmf5mqq3g" --header="Connection: keep-alive" "https://doc-0c-80-docs.googleusercontent.com/docs/securesc/k665ivmiqmum3nhh9vv4ettek14o699p/9d78urp052d6gghgdt3sltlqm345gqfb/1592626950000/00484516897554883881/00484516897554883881/1Z4TyI7FcFVEx8qdl4jO9qxvxaqLSqoEu?e=download&authuser=1&nonce=nacpsmf5mqq3g&user=00484516897554883881&hash=mhlsmlam83ptm57hr8rpgpsdulq2ti0p" -c -O 'rvl-cdip.rar'

--2020-06-20 04:48:11--  https://doc-0c-80-docs.googleusercontent.com/docs/securesc/k665ivmiqmum3nhh9vv4ettek14o699p/9d78urp052d6gghgdt3sltlqm345gqfb/1592626950000/00484516897554883881/00484516897554883881/1Z4TyI7FcFVEx8qdl4jO9qxvxaqLSqoEu?e=download&authuser=1&nonce=nacpsmf5mqq3g&user=00484516897554883881&hash=mhlsmlam83ptm57hr8rpgpsdulq2ti0p
Resolving doc-0c-80-docs.googleusercontent.com (doc-0c-80-docs.googleusercontent.com)... 74.125.203.132, 2404:6800:4008:c03::84
Connecting to doc-0c-80-docs.googleusercontent.com (doc-0c-80-docs.googleusercontent.com)|74.125.203.132|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2020-06-20 04:48:11 ERROR 403: Forbidden.



In [3]:
!gdown --id 1QU2gjy_DcIiBIiy_cN6nx7tYHupUmdeU

Downloading...
From: https://drive.google.com/uc?id=1QU2gjy_DcIiBIiy_cN6nx7tYHupUmdeU
To: /content/rvl-cdip.rar
4.66GB [01:28, 52.5MB/s]


#### Extracting the rar file

In [4]:
get_ipython().system_raw("unrar x rvl-cdip.rar") 

##### Checking the directories

In [5]:
os.listdir()

['.config', 'data_final', 'labels_final.csv', 'rvl-cdip.rar', 'sample_data']

##### Loading the data

In [6]:
import pandas as pd

data = pd.read_csv('labels_final.csv') #reading the csv file

In [8]:
data.head()

,path,label
0,imagesv/v/o/h/voh71d00/509132755+-2755.tif,3
1,imagesl/l/x/t/lxt19d00/502213303.tif,3
2,imagesx/x/e/d/xed05a00/2075325674.tif,2
3,imageso/o/j/b/ojb60d00/517511301+-1301.tif,3
4,imagesq/q/z/k/qzk17e00/2031320195.tif,7


##### Splitting data into train and validation data

In [7]:
from sklearn.model_selection import train_test_split

train_path, validation_path, train_label, validation_label = train_test_split(data['path'], data['label'], test_size=0.2, random_state=42)

In [8]:
len(train_path)

38400

##### Creating a dictionary with labels as keys and category as value

In [9]:
labels_dict = { 0 : 'letter',1: 'form',2: 'email',3 :'handwritten',4 :'advertisement',
                5 : 'scientific report',6 : 'scientific publication',7 :'specification',8 : 'file folder',
                9 : 'news article', 10 : 'budget', 11 : 'invoice',12 : 'presentation',
               13 : 'questionnaire', 14 : 'resume', 15 : 'memo'}

In [10]:
labels_dict.values()

dict_values(['letter', 'form', 'email', 'handwritten', 'advertisement', 'scientific report', 'scientific publication', 'specification', 'file folder', 'news article', 'budget', 'invoice', 'presentation', 'questionnaire', 'resume', 'memo'])

##### Creating train sub folders

In [11]:
for subfolder_name in list(labels_dict.values()):
    os.makedirs(os.path.join('train_images', subfolder_name))

In [12]:
os.listdir('train_images')

['resume',
 'scientific report',
 'memo',
 'news article',
 'invoice',
 'scientific publication',
 'presentation',
 'budget',
 'questionnaire',
 'form',
 'email',
 'handwritten',
 'letter',
 'file folder',
 'specification',
 'advertisement']

In [13]:
#https://thispointer.com/python-how-to-copy-files-from-one-location-to-another-using-shutil-copy/
import shutil
from tqdm import tqdm
for file, label in tqdm(zip(train_path, train_label)):
    shutil.copy('data_final/'+file ,'train_images/'+labels_dict[label]+'/')


38400it [01:34, 407.97it/s]


##### Creating test folders

In [14]:
for subfolder_name in list(labels_dict.values()):
    os.makedirs(os.path.join('validation_images', subfolder_name))

In [15]:
for file, label in tqdm(zip(validation_path, validation_label)):
    shutil.copy('data_final/'+file ,'validation_images/'+labels_dict[label]+'/')

9600it [00:19, 487.00it/s]


In [16]:
dir_path= 'train_images'
for i in os.listdir(dir_path):
    print("No of Images in ",i," category is ",len(os.listdir(os.path.join(dir_path,i))))

No of Images in  resume  category is  2415
No of Images in  scientific report  category is  2379
No of Images in  memo  category is  2401
No of Images in  news article  category is  2392
No of Images in  invoice  category is  2397
No of Images in  scientific publication  category is  2373
No of Images in  presentation  category is  2428
No of Images in  budget  category is  2422
No of Images in  questionnaire  category is  2395
No of Images in  form  category is  2407
No of Images in  email  category is  2379
No of Images in  handwritten  category is  2413
No of Images in  letter  category is  2461
No of Images in  file folder  category is  2351
No of Images in  specification  category is  2391
No of Images in  advertisement  category is  2393


In [17]:
dir_path= 'validation_images'
for i in os.listdir(dir_path):
    print("No of Images in ",i," category is ",len(os.listdir(os.path.join(dir_path,i))))

No of Images in  resume  category is  590
No of Images in  scientific report  category is  620
No of Images in  memo  category is  595
No of Images in  news article  category is  609
No of Images in  invoice  category is  595
No of Images in  scientific publication  category is  612
No of Images in  presentation  category is  578
No of Images in  budget  category is  580
No of Images in  questionnaire  category is  612
No of Images in  form  category is  587
No of Images in  email  category is  614
No of Images in  handwritten  category is  592
No of Images in  letter  category is  554
No of Images in  file folder  category is  652
No of Images in  specification  category is  609
No of Images in  advertisement  category is  601


In [18]:
dir_path= 'train_images'
ImageFlow = tf.keras.preprocessing.image.ImageDataGenerator()
ImageGenerator_train = ImageFlow.flow_from_directory(dir_path,target_size=(400,600),seed=10,batch_size=32)

Found 38397 images belonging to 16 classes.


In [19]:
dir_path='validation_images'
ImageGenerator_validation = ImageFlow.flow_from_directory(dir_path,target_size=(400,600),seed=10,batch_size=32)

Found 9600 images belonging to 16 classes.


In [20]:
IMAGE_SIZE = [224, 224]
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58892288/58889256 [==============================] - 1s 0us/step


In [21]:
train_path = 'train_images'
valid_path = 'validation_images'

In [22]:
for layer in vgg.layers:
  layer.trainable = False

In [23]:
folders = glob('train_images/*')

In [24]:
import os
os.environ['PYTHONHASHSEED'] = '0'

##https://keras.io/getting-started/faq/#how-can-i-obtain-reproducible-results-using-keras-during-development
## Have to clear the session. If you are not clearing, Graph will create again and again and graph size will increses. 
## Varibles will also set to some value from before session
tf.keras.backend.clear_session()

## Set the random seed values to regenerate the model.
np.random.seed(0)
rn.seed(0)

#Input layer
input_layer = Input(shape=(156,256,3),name='Input_Layer')

#Conv Layer
Conv1 = Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='valid',data_format='channels_last',
              activation='relu',kernel_initializer=tf.keras.initializers.he_normal(seed=0),name='Conv1')(input_layer)
#MaxPool Layer
Pool1 = MaxPool2D(pool_size=(2,2),strides=(2,2),padding='valid',data_format='channels_last',name='Pool1')(Conv1)


#Flatten
flatten = Flatten(data_format='channels_last',name='Flatten')(Pool1)

#FC layer
FC1 = Dense(units=30,activation='relu',kernel_initializer=tf.keras.initializers.glorot_normal(seed=32),name='FC1')(flatten)

#FC layer
FC2 = Dense(units=15,activation='relu',kernel_initializer=tf.keras.initializers.glorot_normal(seed=33),name='FC2')(FC1)

#output layer
Out = Dense(units=8,activation='softmax',kernel_initializer=tf.keras.initializers.glorot_normal(seed=3),name='Output')(FC2)

#Creating a model
model = Model(inputs=input_layer,outputs=Out)

In [25]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_Layer (InputLayer)     [(None, 156, 256, 3)]     0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 154, 254, 32)      896       
_________________________________________________________________
Pool1 (MaxPooling2D)         (None, 77, 127, 32)       0         
_________________________________________________________________
Flatten (Flatten)            (None, 312928)            0         
_________________________________________________________________
FC1 (Dense)                  (None, 30)                9387870   
_________________________________________________________________
FC2 (Dense)                  (None, 15)                465       
_________________________________________________________________
Output (Dense)               (None, 8)                 128   

In [26]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01),loss='categorical_crossentropy',metrics=['accuracy'])

In [31]:
model.fit_generator(ImageGenerator_train,steps_per_epoch=len(ImageGenerator_train),epochs=3)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/3


InvalidArgumentError: ignored